In [1]:
import pandas as pd
import numpy as np
import requests
import re
import csv

### GGF HH production

In [2]:
pythia_fragment_CP5 = """
from Configuration.Generator.Pythia8CommonSettings_cfi import *
from Configuration.Generator.MCTunes2017.PythiaCP5Settings_cfi import *
from Configuration.Generator.PSweightsPythia.PythiaPSweightsSettings_cfi import *

generator = cms.EDFilter("Pythia8HadronizerFilter",
                         maxEventsToPrint = cms.untracked.int32(1),
                         pythiaPylistVerbosity = cms.untracked.int32(1),
                         filterEfficiency = cms.untracked.double(1.0),
                         pythiaHepMCVerbosity = cms.untracked.bool(False),
                         comEnergy = cms.double(13000.),
                         PythiaParameters = cms.PSet(
        pythia8CommonSettingsBlock,
        pythia8CP5SettingsBlock,
        pythia8PSweightsSettingsBlock,
        processParameters = cms.vstring(__CHANNEL_DECAY_FRAGMENT__),
        parameterSets = cms.vstring('pythia8CommonSettings',
                                    'pythia8CP5Settings',
                                    'processParameters',
                                    'pythia8PSweightsSettings'
                                    )
        )
                         )

ProductionFilterSequence = cms.Sequence(generator)
"""

In [3]:
def fragmentsDictCreator (decay_fr):
    dict = { '2016': pythia_fragment_CP5.replace('__CHANNEL_DECAY_FRAGMENT__',decay_fr),
             '2016APV': pythia_fragment_CP5.replace('__CHANNEL_DECAY_FRAGMENT__',decay_fr),
             '2017': pythia_fragment_CP5.replace('__CHANNEL_DECAY_FRAGMENT__',decay_fr),
             '2018': pythia_fragment_CP5.replace('__CHANNEL_DECAY_FRAGMENT__',decay_fr)
                    }
    return dict

In [4]:
LHEproducer = """
import FWCore.ParameterSet.Config as cms

# link to cards:
# __EXAMPLE__

externalLHEProducer = cms.EDProducer('ExternalLHEProducer',
    args = cms.vstring('__GRIDPACK__'),
    nEvents = cms.untracked.uint32(5000),
    numberOfParameters = cms.uint32(1),
    outputFile = cms.string('cmsgrid_final.lhe'),
    scriptName = cms.FileInPath('GeneratorInterface/LHEInterface/data/run_generic_tarball_cvmfs.sh')
)
"""

In [5]:
years = ["2016", "2016APV", "2017", "2018"]
processes = ["Radion","BulkGraviton"]
decays = ["2G2WTo2G2Q1L1Nu","2G2WTo2G2l2Nu","2G2WTo2G4Q","2G2ZTo2G4Q"]
mass_point = ["250","260","270","280","300","320","350","400","450","500","550","600","650","700","750","800","850","900","1000"]
tot_events = [800000]*19
gridpacks_dict = {}
example_dict = {}
dataset_names = {}

In [6]:
for year in range(len(years)):
    t_example1 = 'https://github.com/cms-sw/genproductions/tree/master/bin/MadGraph5_aMCatNLO/cards/production/2017/13TeV/exo_diboson/Spin-0'
    t_example2 = 'https://github.com/cms-sw/genproductions/tree/master/bin/MadGraph5_aMCatNLO/cards/production/2017/13TeV/exo_diboson/Spin-2'
    t_datasetname_year = 'GluGluTo{process}ToHHTo{decay}_M-{mass}_TuneCP5_PSWeights_narrow_13TeV-madgraph-pythia8'
    t_gp_year = "/cvmfs/cms.cern.ch/phys_generator/gridpacks/UL/13TeV/madgraph/V5_2.6.5/GF_Spin_{spin}/{process}_hh{graviton}narrow_M{mass}/v1/{process}_hh{graviton}narrow_M{mass}_slc7_amd64_gcc700_CMSSW_10_6_19_tarball.tar.xz"    
    tmp_dataset_dict={}
    tmp_gridpack_dict={}
    tmp_example_dict={}
    dataset_names_year = []
    gp_ggf_year = "gp_ggf_" + years[year]
    gp_ggf_year = []
    

    for process in range(len(processes)):
        #gp_ggf_year = "gp_ggf_" + years[year]
        dataset_names_year = []
        gp_ggf_year = []
        tmp_example_dict[processes[process]]={}
        tmp_dataset_dict[processes[process]]={}
        tmp_gridpack_dict[processes[process]]={}
        for decay in decays:
            dataset_names_year = []
            gp_ggf_year = []
            for mass in mass_point:
                if processes[process] == "Radion":
                    d = {"process":processes[process], "decay": decay, "mass": mass, "spin": "0", "graviton": "_"}
                    tmp_example_dict[processes[process]][decay] = t_example1
                elif processes[process] == "BulkGraviton":
                    d = {"process":processes[process], "decay": decay, "mass": mass, "spin": "2", "graviton": "_GF_HH_"}
                    tmp_example_dict[processes[process]][decay] = t_example2
                gp_ggf_year.append(t_gp_year.format_map(d))
                dataset_names_year.append(t_datasetname_year.format_map(d))
                tmp_dataset_dict[processes[process]][decay] = dataset_names_year
                tmp_gridpack_dict[processes[process]][decay] = gp_ggf_year
    gridpacks_dict[years[year]] = tmp_gridpack_dict
    dataset_names[years[year]] = tmp_dataset_dict
    example_dict[years[year]] = tmp_example_dict
#print(example_dict['2016']['Radion'])
#gridpacks_dict = {'VBF_HH' : {'2016': gp_vbf_2016, '2017': gp_vbf_2017, '2018': gp_vbf_2018}}                                                                                                              
#for x in gridpacks_dict:                                                                                                                                                                                   
#    print (x)                                                                                                                                                                                              
#    for y in gridpacks_dict[x]:                                                                                                                                                                            
#        print (y,':', gridpacks_dict[x][y])                 

In [7]:
ggf_hh_2G2WTo2G2Q1L1Nu = '''                                                                                                                                                                                           
    '15:onMode = on', # allow all tau decays. Leptonic and Hadronic 
    '24:mMin = 0.05', # the lower limit of the allowed mass range generated by the Breit-Wigner (in GeV)
    '24:onMode = off', # Turn off all W decays 
    '24:onIfAny = 1 2 3 4 5 11 13 15', # Add W->enu, W->munu, W->taunu. Add W->qq decays 
    '25:onMode = off', # Turn off all H decays 
    '25:onIfMatch = 22 22', # Add H->gg decay
    '25:onIfMatch = 24 -24', # Add H->WW decay
    'ResonanceDecayFilter:filter = on',
    'ResonanceDecayFilter:exclusive = on', #off: require at least the specified number of daughters, on: require exactly the specified number of daughters
    'ResonanceDecayFilter:eMuTauAsEquivalent = on', #on: treat electrons, muons , and taus as equivalent
    'ResonanceDecayFilter:allNuAsEquivalent  = on', #on: treat all three neutrino flavours as equivalent
    'ResonanceDecayFilter:udscbAsEquivalent  = on',  #on: treat udscb quarks as equivalent
    'ResonanceDecayFilter:mothers = 25,24', #list of mothers not specified -> count all particles in hard process+resonance decays (better to avoid specifying mothers when including leptons from the lhe in counting, since intermediate resonances are not gauranteed to appear in general
    'ResonanceDecayFilter:daughters = 1,1,11,12,22,22', # qq,lnu,gg
'''

ggf_hh_2G2WTo2G2l2Nu = '''
    '15:onMode = on', # allow all tau decays. Leptonic and Hadronic 
    '24:mMin = 0.05', # the lower limit of the allowed mass range generated by the Breit-Wigner (in GeV)
    '24:onMode = off', # Turn off all W decays 
    '24:onIfAny = 11 13 15', # Add W->enu, W->munu, W->taunu 
    '25:onMode = off', # Turn off all H decays 
    '25:onIfMatch = 22 22', # Add H->gg decay
    '25:onIfMatch = 24 -24', # Add H->WW decay
    'ResonanceDecayFilter:filter = on',
    'ResonanceDecayFilter:exclusive = on', #off: require at least the specified number of daughters, on: require exactly the specified number of daughters
    'ResonanceDecayFilter:eMuTauAsEquivalent = on', #on: treat electrons, muons , and taus as equivalent
    'ResonanceDecayFilter:allNuAsEquivalent  = on', #on: treat all three neutrino flavours as equivalent
    'ResonanceDecayFilter:udscbAsEquivalent  = on',  #on: treat udscb quarks as equivalent
    'ResonanceDecayFilter:mothers = 25,24', #list of mothers not specified -> count all particles in hard process+resonance decays (better to avoid specifying mothers when including leptons from the lhe in counting, since intermediate resonances are not gauranteed to appear in general
    'ResonanceDecayFilter:daughters = 11,12,11,12,22,22', # lnu,lnu,gg 
'''

ggf_hh_2G2WTo2G4Q = '''
    '24:mMin = 0.05', # the lower limit of the allowed mass range generated by the Breit-Wigner (in GeV)
    '24:onMode = off', # Turn off all W decays 
    '24:onIfAny = 1 2 3 4 5', # Add W->qq decays 
    '25:onMode = off', # Turn off all H decays 
    '25:onIfMatch = 22 22', # Add H->gg decay
    '25:onIfMatch = 24 -24', # Add H->WW decay 
    'ResonanceDecayFilter:filter = on',
    'ResonanceDecayFilter:exclusive = on', #off: require at least the specified number of daughters, on: require exactly the specified number of daughters
    'ResonanceDecayFilter:udscbAsEquivalent  = on',  #on: treat udscb quarks as equivalent
    'ResonanceDecayFilter:mothers = 25,24', #list of mothers not specified -> count all particles in hard process+resonance decays (better to avoid specifying mothers when including leptons from the lhe in counting, since intermediate resonances are not gauranteed to appear in general
    'ResonanceDecayFilter:daughters = 1,1,1,1,22,22', # qq,qq,gg 
'''

ggf_hh_2G2ZTo2G4Q = '''
    '23:mMin = 0.05', # the lower limit of the allowed mass range generated by the Breit-Wigner (in GeV)
    '23:onMode = off', # Turn off all Z decays 
    '23:onIfAny = 1 2 3 4 5', # Add Z->qq decays 
    '25:onMode = off', # Turn off all H decays 
    '25:onIfMatch = 22 22', # Add H->gg decay
    '25:onIfMatch = 23 23', # Add H->ZZ decay 
    'ResonanceDecayFilter:filter = on',
    'ResonanceDecayFilter:exclusive = on', #off: require at least the specified number of daughters, on: require exactly the specified number of daughters
    'ResonanceDecayFilter:udscbAsEquivalent  = on',  #on: treat udscb quarks as equivalent
    'ResonanceDecayFilter:mothers = 25,23', #list of mothers not specified -> count all particles in hard process+resonance decays (better to avoid specifying mothers when including leptons from the lhe in counting, since intermediate resonances are not gauranteed to appear in general
    'ResonanceDecayFilter:daughters = 1,1,1,1,22,22', # qq,qq,gg 
'''

## .csv Production

In [8]:
#MadgraphVersion = re.compile("V5_2\.[0-9]\.[0-9]")
process_pythia_map = {}
tmp_process_pythia_map = {}
for process in processes:
    tmp_process_pythia_map[process] = {}
    for decay in decays:
        if decay == "2G2WTo2G2Q1L1Nu":
            tmp_process_pythia_map[process][decay] = fragmentsDictCreator(ggf_hh_2G2WTo2G2Q1L1Nu)
        elif decay == "2G2WTo2G2l2Nu":
            tmp_process_pythia_map[process][decay] = fragmentsDictCreator(ggf_hh_2G2WTo2G2l2Nu)
        elif decay == "2G2WTo2G4Q":
            tmp_process_pythia_map[process][decay] = fragmentsDictCreator(ggf_hh_2G2WTo2G4Q)
        elif decay == "2G2ZTo2G4Q":
            tmp_process_pythia_map[process][decay] = fragmentsDictCreator(ggf_hh_2G2ZTo2G4Q)
process_pythia_map.update(tmp_process_pythia_map)

In [9]:
#for year in range(len(years)):
decay_mode = ["SL","FL","FH","FHZZ"]
for year in gridpacks_dict:
    #print(year)
    for process in processes:
        for dm in decay_mode:
            with open('ggF_'+ process + '_' + dm + '_'+year+'.csv', 'w') as csvfile:
                csvwriter = csv.writer(csvfile, delimiter=',',
                            quotechar='"', quoting=csv.QUOTE_MINIMAL)
                csvwriter.writerow(['Dataset name','Events', 'fragment','notes','Generator','mcdbid','time','size'])
                for i in range(len(mass_point)):
                    tmp_fragment=""
                    #print(gridpacks_dict[year][i])
                    #print(dataset_names[year][i])
                    #version = MadgraphVersion.search(path).group(0)
                    version="2.6.5"
                    if year == "2016APV":
                        events = round(tot_events[i]*0.54)
                    elif year == "2016":
                        events = round(tot_events[i]*0.46)
                    elif year == "2017":
                        events = tot_events[i]
                    elif year == "2018":
                        events = tot_events[i]
                        
                    if dm == "SL":
                        dataset_name = dataset_names[year][process]["2G2WTo2G2Q1L1Nu"][i]
                        tmp_fragment = LHEproducer.replace('__GRIDPACK__',gridpacks_dict[year][process]["2G2WTo2G2Q1L1Nu"][i]) + '\n' + process_pythia_map[process]["2G2WTo2G2Q1L1Nu"][year]
                        note = dataset_name.replace('_',' ')
                    elif dm == "FL":
                        dataset_name = dataset_names[year][process]["2G2WTo2G2l2Nu"][i]
                        tmp_fragment = LHEproducer.replace('__GRIDPACK__',gridpacks_dict[year][process]["2G2WTo2G2l2Nu"][i]) + '\n' + process_pythia_map[process]["2G2WTo2G2l2Nu"][year]
                        note = dataset_name.replace('_',' ')
                    elif dm == "FH":
                        dataset_name = dataset_names[year][process]["2G2WTo2G4Q"][i]
                        tmp_fragment = LHEproducer.replace('__GRIDPACK__',gridpacks_dict[year][process]["2G2WTo2G4Q"][i]) + '\n' + process_pythia_map[process]["2G2WTo2G4Q"][year]
                        note = dataset_name.replace('_',' ')
                    elif dm == "FHZZ":
                        dataset_name = dataset_names[year][process]["2G2ZTo2G4Q"][i]
                        tmp_fragment = LHEproducer.replace('__GRIDPACK__',gridpacks_dict[year][process]["2G2ZTo2G4Q"][i]) + '\n' + process_pythia_map[process]["2G2ZTo2G4Q"][year]
                        note = dataset_name.replace('_',' ')
                    final_fragment = tmp_fragment.replace('__EXAMPLE__',example_dict[year][process]["2G2WTo2G2Q1L1Nu"])
                    generators="Madgraph_" + version + "  Pythia8"
                    mcdb_id = '0'
                    time = '0.001'
                    size = '30'
                    csvwriter.writerow([dataset_name, events, final_fragment, note, generators, mcdb_id, time, size])